# 📊 样本量计算与功效分析

---

## 🎬 真实场景：一个让 PM 抓狂的问题

周一早会，产品经理小王问你：

> 「我们想测试新的推荐算法，预期能提升点击率 2%。实验要跑多久？需要多少用户？」

你挠了挠头：「这个...让我算算...」

**两周后**

实验结束，结果显示 p-value = 0.15，「不显著」。

小王急了：「明明用户反馈很好啊！是不是样本量不够？」

你尴尬地发现：**根本没有提前做功效分析！**

---

### 样本量计算的重要性

| 问题 | 后果 |
|------|------|
| 样本量太小 | 检测不出真实效应 → 错失好策略 |
| 样本量太大 | 浪费资源、延长实验周期 |
| 不提前计算 | 实验结束后才发现问题 |

**功效分析就是在实验前回答：我需要多少样本才能检测出预期的效应？**

---

## 📚 本节学习目标

完成本练习后，你将能够：

1. 🎯 理解 **第一类错误 (α)** 和 **第二类错误 (β)** 的含义
2. 🎯 掌握 **MDE（最小可检测效应）** 的概念
3. 🎯 推导并实现 **样本量计算公式**
4. 🎯 绘制 **功效曲线**，进行敏感性分析
5. 🎯 了解 **序贯分析** 的基本思想

> ⚠️ **重要**: 这是 A/B 测试的基础！每个实验都应该提前做功效分析。

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import norm, t
from typing import Tuple, Dict, Optional
import warnings
warnings.filterwarnings('ignore')

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')
np.random.seed(42)

print("✅ 环境准备完毕！")

---

## 🧠 Part 1: 假设检验的两类错误

### 1.1 问题设定

在 A/B 测试中，我们要回答一个问题：

> 新策略 (B) 真的比旧策略 (A) 好吗？

统计学上，我们设定假设：

- **H₀ (原假设)**: $\mu_B - \mu_A = 0$ （没有效应）
- **H₁ (备择假设)**: $\mu_B - \mu_A \neq 0$ （有效应）

### 1.2 两类错误

| | 真相: H₀ 为真 (无效应) | 真相: H₁ 为真 (有效应) |
|---|---|---|
| **决策: 拒绝 H₀** | ❌ **第一类错误 (α)** | ✅ 正确决策 (Power) |
| **决策: 不拒绝 H₀** | ✅ 正确决策 (1-α) | ❌ **第二类错误 (β)** |

**第一类错误 (α, Type I Error)**:
- 实际没有效应，但我们错误地认为有效应
- 又叫「假阳性」、「误报」
- 通常设为 α = 0.05

**第二类错误 (β, Type II Error)**:
- 实际有效应，但我们错误地认为没有效应
- 又叫「假阴性」、「漏报」
- 通常设为 β = 0.2

**功效 (Power)**:
- Power = 1 - β
- 当真实效应存在时，能正确检测出来的概率
- 通常要求 Power ≥ 0.8 (80%)

In [ ]:
def visualize_hypothesis_testing(
    effect_size: float = 0.5,
    alpha: float = 0.05,
    n_per_group: int = 100
):
    """
    可视化假设检验的两类错误
    """
    # 标准误
    se = np.sqrt(2 / n_per_group)  # 假设方差为 1
    
    # 临界值
    z_alpha = norm.ppf(1 - alpha / 2)
    critical_value = z_alpha * se
    
    # 绘图
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.linspace(-1, 1.5, 1000)
    
    # H0 下的分布 (centered at 0)
    y_h0 = norm.pdf(x, 0, se)
    ax.plot(x, y_h0, 'b-', linewidth=2, label='H₀: 无效应')
    ax.fill_between(x[x > critical_value], y_h0[x > critical_value], 
                   alpha=0.3, color='red', label=f'α/2 = {alpha/2:.3f} (第一类错误)')
    ax.fill_between(x[x < -critical_value], y_h0[x < -critical_value], 
                   alpha=0.3, color='red')
    
    # H1 下的分布 (centered at effect_size)
    y_h1 = norm.pdf(x, effect_size, se)
    ax.plot(x, y_h1, 'g-', linewidth=2, label=f'H₁: 效应 = {effect_size}')
    
    # β 区域 (H1 下，未拒绝 H0 的区域)
    mask_beta = (x > -critical_value) & (x < critical_value)
    ax.fill_between(x[mask_beta], y_h1[mask_beta], 
                   alpha=0.3, color='orange', label='β (第二类错误)')
    
    # 临界值线
    ax.axvline(critical_value, color='black', linestyle='--', linewidth=1.5)
    ax.axvline(-critical_value, color='black', linestyle='--', linewidth=1.5)
    
    # 计算 β 和 Power
    beta = norm.cdf(critical_value, effect_size, se) - norm.cdf(-critical_value, effect_size, se)
    power = 1 - beta
    
    ax.set_xlabel('效应估计', fontsize=12)
    ax.set_ylabel('概率密度', fontsize=12)
    ax.set_title(f'假设检验的两类错误\nn={n_per_group}/组, α={alpha}, β={beta:.3f}, Power={power:.3f}', fontsize=14)
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 统计量:")
    print(f"   样本量 (每组): {n_per_group}")
    print(f"   真实效应: {effect_size}")
    print(f"   显著性水平 α: {alpha}")
    print(f"   第二类错误 β: {beta:.4f}")
    print(f"   功效 (Power): {power:.4f}")

visualize_hypothesis_testing(effect_size=0.3, alpha=0.05, n_per_group=100)

### 1.3 α 和 β 的权衡

在固定样本量下，α 和 β 存在权衡：

- 降低 α（更严格）→ β 增加 → Power 降低
- 降低 β（更高 Power）→ α 增加 → 假阳性增加

**增加样本量可以同时降低 α 和 β！**

In [ ]:
# 交互式探索：改变样本量
print("改变样本量对 Power 的影响:")
print("=" * 50)

effect_size = 0.3
alpha = 0.05

for n in [50, 100, 200, 500, 1000]:
    se = np.sqrt(2 / n)
    z_alpha = norm.ppf(1 - alpha / 2)
    critical_value = z_alpha * se
    beta = norm.cdf(critical_value, effect_size, se) - norm.cdf(-critical_value, effect_size, se)
    power = 1 - beta
    print(f"n = {n:4d} → Power = {power:.3f} {'✅' if power >= 0.8 else '❌'}")

---

## 📐 Part 2: MDE (最小可检测效应)

### 2.1 什么是 MDE？

**MDE (Minimum Detectable Effect)** 是在给定样本量、α、Power 下，能检测出的最小效应。

换句话说：如果真实效应小于 MDE，你的实验就「看不见」它。

### 2.2 MDE 公式

对于双样本 t 检验（假设方差相等）：

$$MDE = (z_{1-\alpha/2} + z_{1-\beta}) \cdot \sqrt{\frac{2\sigma^2}{n}}$$

其中：
- $z_{1-\alpha/2}$: 对应 α 的 z 值（双边检验）
- $z_{1-\beta}$: 对应 Power 的 z 值
- $\sigma$: 总体标准差
- $n$: 每组样本量

### 2.3 业务理解

MDE 需要结合业务来理解：

| 场景 | 当前指标 | 预期提升 | MDE 要求 |
|------|---------|---------|----------|
| 点击率优化 | 3% | 相对提升 10% | MDE ≤ 0.3% |
| 转化率优化 | 5% | 相对提升 5% | MDE ≤ 0.25% |
| 人均 GMV | ¥200 | 提升 ¥5 | MDE ≤ ¥5 |

In [ ]:
def calculate_mde(
    n_per_group: int,
    sigma: float = 1.0,
    alpha: float = 0.05,
    power: float = 0.8
) -> float:
    """
    计算最小可检测效应 (MDE)
    
    参数:
        n_per_group: 每组样本量
        sigma: 总体标准差
        alpha: 显著性水平
        power: 功效 (1 - β)
    
    返回:
        MDE: 最小可检测效应
    """
    z_alpha = norm.ppf(1 - alpha / 2)  # 双边检验
    z_beta = norm.ppf(power)
    
    mde = (z_alpha + z_beta) * np.sqrt(2 * sigma**2 / n_per_group)
    
    return mde

# 示例
print("📏 MDE 计算示例")
print("=" * 50)
print(f"假设 σ = 1, α = 0.05, Power = 0.8")
print()

for n in [100, 500, 1000, 5000, 10000]:
    mde = calculate_mde(n)
    print(f"n = {n:5d} → MDE = {mde:.4f} ({mde*100:.2f}% of σ)")

---

## 🔢 Part 3: 样本量计算

### 3.1 核心公式推导

从 MDE 公式反推样本量：

$$n = \frac{2\sigma^2 (z_{1-\alpha/2} + z_{1-\beta})^2}{\delta^2}$$

其中 $\delta$ 是预期的效应大小（MDE）。

### 3.2 连续结果的样本量

对于连续结果（如人均消费）：

$$n = \frac{2\sigma^2 (z_{1-\alpha/2} + z_{1-\beta})^2}{(\mu_B - \mu_A)^2}$$

### 3.3 二元结果的样本量

对于二元结果（如点击/不点击，转化/不转化）：

$$n = \frac{2 \cdot \bar{p}(1-\bar{p}) \cdot (z_{1-\alpha/2} + z_{1-\beta})^2}{(p_B - p_A)^2}$$

其中 $\bar{p} = (p_A + p_B)/2$ 是合并比例。

# 📖 示范：实现连续结果的样本量计算

def sample_size_continuous(
    effect_size: float,
    sigma: float,
    alpha: float = 0.05,
    power: float = 0.8
) -> int:
    """
    计算连续结果的每组所需样本量
    
    参数:
        effect_size: 预期效应大小 (μ_B - μ_A)
        sigma: 总体标准差
        alpha: 显著性水平
        power: 功效
    
    返回:
        n: 每组所需样本量
    """
    # 步骤 1: 计算 z 分数
    z_alpha = norm.ppf(1 - alpha / 2)  # 双边检验，每侧 alpha/2
    z_beta = norm.ppf(power)            # Power = 1 - β
    
    # 步骤 2: 应用公式
    # n = 2 * σ² * (z_α/2 + z_β)² / δ²
    n = 2 * sigma**2 * (z_alpha + z_beta)**2 / effect_size**2
    
    return int(np.ceil(n))

# 数学推导说明
print("📐 公式推导")
print("=" * 60)
print("在双样本 t 检验中：")
print("  - 标准误: SE = σ√(2/n)")
print("  - 检验统计量: t = (μ̂₁ - μ̂₀) / SE")
print()
print("要达到 Power = 1 - β，需要：")
print("  δ / SE = z_{1-α/2} + z_{1-β}")
print()
print("代入 SE 并求解 n：")
print("  n = 2σ² (z_{1-α/2} + z_{1-β})² / δ²")
print()
print("关键点:")
print("  1. 双边检验用 alpha/2")
print("  2. z_beta = norm.ppf(power)，不是 norm.ppf(1-power)")
print("  3. 效应在分母，所以 n ∝ 1/δ²（效应减半，样本量翻4倍）")

In [ ]:
# 🎯 练习：实现二元结果的样本量计算

def sample_size_proportion(
    p_control: float,
    p_treatment: float,
    alpha: float = 0.05,
    power: float = 0.8
) -> int:
    """
    计算二元结果的每组所需样本量
    
    参数:
        p_control: 对照组比例
        p_treatment: 处理组比例
        alpha: 显著性水平
        power: 功效
    
    返回:
        n: 每组所需样本量
    """
    # TODO: 实现公式
    # 提示: 
    # 1. 计算合并比例 p_bar = (p_control + p_treatment) / 2
    # 2. 计算效应大小 effect = p_treatment - p_control
    # 3. n = 2 * p_bar * (1 - p_bar) * (z_alpha + z_beta)^2 / effect^2
    
    z_alpha = None  # 👈 你的代码
    z_beta = None   # 👈 你的代码
    
    p_bar = None    # 👈 你的代码
    effect = None   # 👈 你的代码
    
    n = None  # 👈 你的代码
    
    if n is None:
        return None
    
    return int(np.ceil(n))


# ============================================================
# 📝 参考答案（折叠）
# ============================================================

# <details>
# <summary>点击查看答案</summary>

"""
【解题思路】

对于比例 p，方差为 Var(p) = p(1-p)

在 H₀ 下，两组比例相等，使用合并估计：
p̄ = (p₁ + p₂) / 2

标准误为：SE = √[2p̄(1-p̄)/n]

【完整代码】
"""

def sample_size_proportion_answer(
    p_control: float,
    p_treatment: float,
    alpha: float = 0.05,
    power: float = 0.8
) -> int:
    # 步骤 1: 获取 z 分数
    z_alpha = norm.ppf(1 - alpha / 2)
    z_beta = norm.ppf(power)
    
    # 步骤 2: 计算合并比例
    p_bar = (p_control + p_treatment) / 2
    
    # 步骤 3: 计算效应大小
    effect = p_treatment - p_control
    
    # 步骤 4: 应用公式
    # n = 2 * p̄(1-p̄) * (z_α/2 + z_β)² / (p₁ - p₀)²
    n = 2 * p_bar * (1 - p_bar) * (z_alpha + z_beta)**2 / effect**2
    
    return int(np.ceil(n))

"""
【为什么用合并比例？】

在 H₀: p₁ = p₀ 下，两组的真实比例相同，应该使用合并的估计。
这被称为"pooled variance"估计。

【数值示例】
p_control = 0.10, p_treatment = 0.12
effect = 0.02 (2% 的绝对提升)
p_bar = 0.11
Var(p̄) = 0.11 * 0.89 = 0.0979

α = 0.05, Power = 0.8
→ n ≈ 3,841 / 组

【业务解释】
转化率从 10% 提升到 12%（相对提升 20%），
需要约 3,800 用户/组 = 7,600 总样本
才能以 80% 的把握检测出来。

【常见错误】
❌ 使用 p_control * (1 - p_control) 作为方差
✅ 应该用合并的 p_bar * (1 - p_bar)
"""

# </details>

In [ ]:
# 📝 参考答案
# ================================

def sample_size_continuous_solution(
    effect_size: float,
    sigma: float,
    alpha: float = 0.05,
    power: float = 0.8
) -> int:
    z_alpha = norm.ppf(1 - alpha / 2)
    z_beta = norm.ppf(power)
    
    n = 2 * sigma**2 * (z_alpha + z_beta)**2 / effect_size**2
    
    return int(np.ceil(n))


def sample_size_proportion_solution(
    p_control: float,
    p_treatment: float,
    alpha: float = 0.05,
    power: float = 0.8
) -> int:
    z_alpha = norm.ppf(1 - alpha / 2)
    z_beta = norm.ppf(power)
    
    p_bar = (p_control + p_treatment) / 2
    effect = p_treatment - p_control
    
    n = 2 * p_bar * (1 - p_bar) * (z_alpha + z_beta)**2 / effect**2
    
    return int(np.ceil(n))

In [ ]:
# 测试样本量计算
print("📊 样本量计算示例")
print("=" * 60)

# 场景 1: 连续结果（人均消费）
print("\n场景 1: 人均消费提升")
print("-" * 40)
print("当前人均消费: ¥200, 标准差: ¥80")
print("预期提升: ¥10")
n1 = sample_size_continuous_solution(effect_size=10, sigma=80)
print(f"所需样本量 (每组): {n1:,}")
print(f"总样本量: {2*n1:,}")

# 场景 2: 二元结果（点击率）
print("\n场景 2: 点击率提升")
print("-" * 40)
print("当前点击率: 5%")
print("预期提升到: 5.5% (相对提升 10%)")
n2 = sample_size_proportion_solution(p_control=0.05, p_treatment=0.055)
print(f"所需样本量 (每组): {n2:,}")
print(f"总样本量: {2*n2:,}")

# 场景 3: 转化率
print("\n场景 3: 转化率提升")
print("-" * 40)
print("当前转化率: 2%")
print("预期提升到: 2.2% (相对提升 10%)")
n3 = sample_size_proportion_solution(p_control=0.02, p_treatment=0.022)
print(f"所需样本量 (每组): {n3:,}")
print(f"总样本量: {2*n3:,}")

---

## 📈 Part 4: 功效曲线

功效曲线展示：在固定样本量下，不同效应大小对应的功效。

这对于理解实验的「检测能力」非常重要。

In [ ]:
def calculate_power(
    n_per_group: int,
    effect_size: float,
    sigma: float = 1.0,
    alpha: float = 0.05
) -> float:
    """
    计算功效
    """
    se = np.sqrt(2 * sigma**2 / n_per_group)
    z_alpha = norm.ppf(1 - alpha / 2)
    
    # 非中心参数
    ncp = effect_size / se
    
    # 功效 = P(拒绝 H0 | H1 为真)
    power = 1 - norm.cdf(z_alpha - ncp) + norm.cdf(-z_alpha - ncp)
    
    return power


def plot_power_curve(
    n_values: list,
    sigma: float = 1.0,
    alpha: float = 0.05
):
    """
    绘制功效曲线
    """
    fig, ax = plt.subplots(figsize=(12, 6))
    
    effect_sizes = np.linspace(0.01, 0.8, 100)
    
    colors = plt.cm.viridis(np.linspace(0, 1, len(n_values)))
    
    for n, color in zip(n_values, colors):
        powers = [calculate_power(n, e, sigma, alpha) for e in effect_sizes]
        ax.plot(effect_sizes, powers, color=color, linewidth=2, 
               label=f'n = {n:,}/组')
    
    # 80% power 参考线
    ax.axhline(0.8, color='red', linestyle='--', linewidth=1.5, 
              label='80% Power')
    
    ax.set_xlabel('效应大小 (Effect Size)', fontsize=12)
    ax.set_ylabel('功效 (Power)', fontsize=12)
    ax.set_title('功效曲线：样本量与效应大小的关系', fontsize=14)
    ax.legend(loc='lower right')
    ax.set_xlim([0, 0.8])
    ax.set_ylim([0, 1])
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 绘制功效曲线
plot_power_curve([100, 200, 500, 1000, 2000])

In [ ]:
# 敏感性分析：不同参数对样本量的影响

def sensitivity_analysis():
    """
    分析不同参数对所需样本量的影响
    """
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    base_effect = 0.2
    base_sigma = 1.0
    base_alpha = 0.05
    base_power = 0.8
    
    # 1. 效应大小的影响
    effects = np.linspace(0.05, 0.5, 50)
    ns_effect = [sample_size_continuous_solution(e, base_sigma, base_alpha, base_power) 
                for e in effects]
    axes[0].plot(effects, ns_effect, 'b-', linewidth=2)
    axes[0].axvline(base_effect, color='red', linestyle='--', alpha=0.7)
    axes[0].set_xlabel('效应大小', fontsize=12)
    axes[0].set_ylabel('所需样本量 (每组)', fontsize=12)
    axes[0].set_title('效应大小 vs 样本量', fontsize=14)
    axes[0].set_yscale('log')
    
    # 2. 标准差的影响
    sigmas = np.linspace(0.5, 2.0, 50)
    ns_sigma = [sample_size_continuous_solution(base_effect, s, base_alpha, base_power) 
               for s in sigmas]
    axes[1].plot(sigmas, ns_sigma, 'g-', linewidth=2)
    axes[1].axvline(base_sigma, color='red', linestyle='--', alpha=0.7)
    axes[1].set_xlabel('标准差', fontsize=12)
    axes[1].set_ylabel('所需样本量 (每组)', fontsize=12)
    axes[1].set_title('标准差 vs 样本量', fontsize=14)
    
    # 3. Power 的影响
    powers = np.linspace(0.6, 0.95, 50)
    ns_power = [sample_size_continuous_solution(base_effect, base_sigma, base_alpha, p) 
               for p in powers]
    axes[2].plot(powers, ns_power, 'm-', linewidth=2)
    axes[2].axvline(base_power, color='red', linestyle='--', alpha=0.7)
    axes[2].set_xlabel('功效 (Power)', fontsize=12)
    axes[2].set_ylabel('所需样本量 (每组)', fontsize=12)
    axes[2].set_title('Power vs 样本量', fontsize=14)
    
    plt.tight_layout()
    plt.show()
    
    print("💡 关键发现:")
    print("   1. 样本量与效应大小的平方成反比（效应减半，样本量翻 4 倍）")
    print("   2. 样本量与标准差的平方成正比（标准差翻倍，样本量翻 4 倍）")
    print("   3. 更高的 Power 需要更大的样本量（但增长较慢）")

sensitivity_analysis()

---

## 🔧 Part 5: 实用工具函数

让我们创建一个完整的样本量计算器。

In [ ]:
class SampleSizeCalculator:
    """
    A/B 测试样本量计算器
    """
    
    def __init__(self, alpha: float = 0.05, power: float = 0.8):
        self.alpha = alpha
        self.power = power
        self.z_alpha = norm.ppf(1 - alpha / 2)
        self.z_beta = norm.ppf(power)
    
    def for_continuous(
        self,
        baseline_mean: float,
        expected_lift: float,
        std: float,
        relative: bool = False
    ) -> Dict:
        """
        连续结果的样本量计算
        
        参数:
            baseline_mean: 基线均值
            expected_lift: 预期提升（绝对值或相对值）
            std: 标准差
            relative: 是否为相对提升（百分比）
        """
        if relative:
            effect = baseline_mean * expected_lift
        else:
            effect = expected_lift
        
        n = 2 * std**2 * (self.z_alpha + self.z_beta)**2 / effect**2
        n = int(np.ceil(n))
        
        return {
            'n_per_group': n,
            'total_n': 2 * n,
            'baseline': baseline_mean,
            'expected_value': baseline_mean + effect,
            'absolute_effect': effect,
            'relative_effect': effect / baseline_mean,
            'mde': effect
        }
    
    def for_proportion(
        self,
        baseline_rate: float,
        expected_lift: float,
        relative: bool = True
    ) -> Dict:
        """
        二元结果的样本量计算
        
        参数:
            baseline_rate: 基线比例（如转化率 0.05）
            expected_lift: 预期提升（如 0.1 表示相对提升 10%）
            relative: 是否为相对提升
        """
        if relative:
            new_rate = baseline_rate * (1 + expected_lift)
        else:
            new_rate = baseline_rate + expected_lift
        
        p_bar = (baseline_rate + new_rate) / 2
        effect = new_rate - baseline_rate
        
        n = 2 * p_bar * (1 - p_bar) * (self.z_alpha + self.z_beta)**2 / effect**2
        n = int(np.ceil(n))
        
        return {
            'n_per_group': n,
            'total_n': 2 * n,
            'baseline_rate': baseline_rate,
            'expected_rate': new_rate,
            'absolute_effect': effect,
            'relative_effect': effect / baseline_rate,
            'mde': effect
        }
    
    def estimate_duration(
        self,
        n_per_group: int,
        daily_traffic: int,
        allocation: float = 1.0
    ) -> Dict:
        """
        估计实验持续时间
        
        参数:
            n_per_group: 每组所需样本量
            daily_traffic: 日均流量
            allocation: 分配到实验的流量比例
        """
        daily_per_group = daily_traffic * allocation / 2
        days = np.ceil(n_per_group / daily_per_group)
        
        return {
            'days': int(days),
            'weeks': days / 7,
            'daily_per_group': daily_per_group
        }

In [ ]:
# 使用示例
calc = SampleSizeCalculator(alpha=0.05, power=0.8)

print("🧮 样本量计算器演示")
print("=" * 60)

# 案例 1: 点击率优化
print("\n📱 案例 1: 推荐算法点击率优化")
print("-" * 40)
result1 = calc.for_proportion(
    baseline_rate=0.03,  # 当前点击率 3%
    expected_lift=0.10,   # 预期相对提升 10%
    relative=True
)
print(f"当前点击率: {result1['baseline_rate']:.1%}")
print(f"预期点击率: {result1['expected_rate']:.1%}")
print(f"相对提升: {result1['relative_effect']:.1%}")
print(f"每组样本量: {result1['n_per_group']:,}")
print(f"总样本量: {result1['total_n']:,}")

# 估计实验时长
duration1 = calc.estimate_duration(
    n_per_group=result1['n_per_group'],
    daily_traffic=100000,  # 日均 10 万用户
    allocation=0.5  # 50% 流量用于实验
)
print(f"预计实验时长: {duration1['days']} 天 ({duration1['weeks']:.1f} 周)")

# 案例 2: 人均消费提升
print("\n💰 案例 2: 促销活动人均消费提升")
print("-" * 40)
result2 = calc.for_continuous(
    baseline_mean=200,    # 当前人均消费 200 元
    expected_lift=0.05,   # 预期相对提升 5%
    std=100,              # 标准差 100 元
    relative=True
)
print(f"当前人均消费: ¥{result2['baseline']:.0f}")
print(f"预期人均消费: ¥{result2['expected_value']:.0f}")
print(f"预期提升: ¥{result2['absolute_effect']:.0f} ({result2['relative_effect']:.1%})")
print(f"每组样本量: {result2['n_per_group']:,}")
print(f"总样本量: {result2['total_n']:,}")

---

## ⚠️ Part 6: 常见陷阱与最佳实践

### 6.1 常见陷阱

| 陷阱 | 后果 | 正确做法 |
|------|------|----------|
| 不提前计算样本量 | 实验后才发现 power 不够 | **必须在实验前做功效分析** |
| 低估方差 | 样本量不足 | 使用历史数据估计方差 |
| 过于乐观的效应预期 | 检测不出真实效应 | 保守估计，考虑多种场景 |
| 忽略多重比较 | 假阳性率膨胀 | 调整 α（Bonferroni 等）|
| 偷看数据 (Peeking) | p-value 失效 | 预设样本量，或用序贯分析 |

In [ ]:
# 演示 Peeking 的问题

def simulate_peeking(
    n_simulations: int = 1000,
    max_n: int = 1000,
    peek_interval: int = 50,
    alpha: float = 0.05
):
    """
    模拟「偷看数据」导致的假阳性率膨胀
    
    真实情况：H0 为真（无效应）
    """
    false_positives_peeking = 0
    false_positives_fixed = 0
    
    for _ in range(n_simulations):
        # 生成无效应数据
        control = np.random.randn(max_n)
        treatment = np.random.randn(max_n)  # 同分布，无效应
        
        # 策略 1: 每隔 peek_interval 看一次，显著就停
        stopped_early = False
        for n in range(peek_interval, max_n + 1, peek_interval):
            _, p = stats.ttest_ind(treatment[:n], control[:n])
            if p < alpha:
                false_positives_peeking += 1
                stopped_early = True
                break
        
        # 策略 2: 固定样本量
        _, p_fixed = stats.ttest_ind(treatment, control)
        if p_fixed < alpha:
            false_positives_fixed += 1
    
    return {
        'peeking_fpr': false_positives_peeking / n_simulations,
        'fixed_fpr': false_positives_fixed / n_simulations,
        'expected_fpr': alpha
    }

# 运行模拟
print("⚠️ Peeking 问题演示")
print("=" * 50)
print("设置: 无效应数据，α = 0.05")
print("每 50 个样本偷看一次，显著就停止")
print()

result = simulate_peeking(n_simulations=1000)
print(f"预期假阳性率: {result['expected_fpr']:.1%}")
print(f"固定样本量: {result['fixed_fpr']:.1%}")
print(f"偷看策略: {result['peeking_fpr']:.1%} ⚠️")
print()
print(f"💡 偷看使假阳性率膨胀了 {result['peeking_fpr']/result['expected_fpr']:.1f} 倍！")

### 6.2 序贯分析简介

如果确实需要在实验过程中做决策，可以使用**序贯分析 (Sequential Analysis)**：

- **Alpha Spending Function**: 预设每次偷看时消耗多少 α
- **Group Sequential Design**: 预设几个固定的分析时间点
- **Always Valid Inference**: 任意时刻停止都有效

这些方法通过调整临界值，保证整体假阳性率不超过 α。

---

## 💡 Part 7: 思考题

### 问题 1: 为什么样本量与效应大小的平方成反比？

**提示**: 想想标准误的公式

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 2: 如果 PM 说「预期提升 5%，但我们也想检测 2% 的提升」，怎么办？

**提示**: 这涉及到 MDE 的选择

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 如果历史数据的方差估计不准确，会有什么影响？如何应对？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: 为什么「偷看数据」会导致假阳性率膨胀？

**提示**: 想想多重比较

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 5: 实际工作中，你会如何向业务方解释「我们需要这么多样本」？

**你的答案:**

*（在这里写下你的思考...）*

---

In [ ]:
class PowerAnalyzer:
    """
    从零实现的功效分析工具
    不依赖 scipy.stats，只使用 numpy
    """
    
    def __init__(self, alpha: float = 0.05, power: float = 0.8):
        """
        参数:
            alpha: 显著性水平 (Type I Error)
            power: 统计功效 (1 - Type II Error)
        """
        self.alpha = alpha
        self.power = power
        
    def _normal_cdf(self, x: float) -> float:
        """
        标准正态分布的累积分布函数 (CDF)
        使用误差函数 erf 的近似
        """
        # 使用 numpy 的 erf (error function)
        # Φ(x) = 0.5 * (1 + erf(x / sqrt(2)))
        return 0.5 * (1 + np.math.erf(x / np.sqrt(2)))
    
    def _normal_ppf(self, p: float, tol: float = 1e-6) -> float:
        """
        标准正态分布的分位数函数 (PPF / Inverse CDF)
        使用二分搜索实现
        
        参数:
            p: 概率 (0 < p < 1)
            tol: 容差
        """
        if p <= 0 or p >= 1:
            raise ValueError("p must be in (0, 1)")
        
        # 二分搜索
        low, high = -10, 10
        while high - low > tol:
            mid = (low + high) / 2
            if self._normal_cdf(mid) < p:
                low = mid
            else:
                high = mid
        
        return (low + high) / 2
    
    def _get_z_values(self):
        """获取 z 分数"""
        z_alpha = self._normal_ppf(1 - self.alpha / 2)  # 双边检验
        z_beta = self._normal_ppf(self.power)
        return z_alpha, z_beta
    
    def sample_size_continuous(
        self, 
        effect_size: float,
        std: float,
        ratio: float = 1.0
    ) -> dict:
        """
        计算连续结果的样本量
        
        参数:
            effect_size: 预期效应大小
            std: 标准差
            ratio: 实验组/对照组的比例 (默认 1:1)
        
        返回:
            字典，包含详细结果
        """
        z_alpha, z_beta = self._get_z_values()
        
        # 公式: n1 = (1 + 1/r) * (z_α + z_β)² * σ² / δ²
        # 其中 r = n1/n0 (比例)
        n_control = ((1 + 1/ratio) * (z_alpha + z_beta)**2 * 
                     std**2 / effect_size**2)
        n_treatment = n_control * ratio
        
        return {
            'n_control': int(np.ceil(n_control)),
            'n_treatment': int(np.ceil(n_treatment)),
            'total_n': int(np.ceil(n_control + n_treatment)),
            'effect_size': effect_size,
            'std': std,
            'alpha': self.alpha,
            'power': self.power,
            'z_alpha': z_alpha,
            'z_beta': z_beta
        }
    
    def sample_size_proportion(
        self,
        p_control: float,
        p_treatment: float,
        ratio: float = 1.0
    ) -> dict:
        """
        计算比例检验的样本量
        
        参数:
            p_control: 对照组比例
            p_treatment: 实验组比例
            ratio: 实验组/对照组的比例
        """
        z_alpha, z_beta = self._get_z_values()
        
        # 合并比例
        p_bar = (p_control + ratio * p_treatment) / (1 + ratio)
        
        # 公式推导自 arcsin 变换
        # 简化版本（适用于 p 不太接近 0 或 1）
        pooled_var = p_bar * (1 - p_bar)
        effect = p_treatment - p_control
        
        n_control = ((1 + 1/ratio) * (z_alpha + z_beta)**2 * 
                     pooled_var / effect**2)
        n_treatment = n_control * ratio
        
        return {
            'n_control': int(np.ceil(n_control)),
            'n_treatment': int(np.ceil(n_treatment)),
            'total_n': int(np.ceil(n_control + n_treatment)),
            'p_control': p_control,
            'p_treatment': p_treatment,
            'effect_size': effect,
            'relative_lift': effect / p_control,
            'alpha': self.alpha,
            'power': self.power
        }
    
    def calculate_power(
        self,
        n_per_group: int,
        effect_size: float,
        std: float = 1.0
    ) -> float:
        """
        给定样本量，计算功效
        
        参数:
            n_per_group: 每组样本量
            effect_size: 效应大小
            std: 标准差
        """
        z_alpha, _ = self._get_z_values()
        
        # 标准误
        se = std * np.sqrt(2 / n_per_group)
        
        # 非中心参数
        ncp = effect_size / se
        
        # Power = Φ(ncp - z_α) + Φ(-ncp - z_α)
        # 对于双边检验，主要是第一项
        power = 1 - self._normal_cdf(z_alpha - ncp)
        
        return power
    
    def calculate_mde(
        self,
        n_per_group: int,
        std: float = 1.0
    ) -> float:
        """
        计算最小可检测效应 (MDE)
        
        参数:
            n_per_group: 每组样本量
            std: 标准差
        """
        z_alpha, z_beta = self._get_z_values()
        
        mde = (z_alpha + z_beta) * std * np.sqrt(2 / n_per_group)
        
        return mde
    
    def simulate_experiment(
        self,
        n_per_group: int,
        true_effect: float,
        std: float = 1.0,
        n_simulations: int = 10000
    ) -> dict:
        """
        蒙特卡洛模拟验证功效
        
        参数:
            n_per_group: 每组样本量
            true_effect: 真实效应
            std: 标准差
            n_simulations: 模拟次数
        
        返回:
            模拟结果统计
        """
        z_alpha, _ = self._get_z_values()
        rejections = 0
        
        for _ in range(n_simulations):
            # 生成数据
            control = np.random.normal(0, std, n_per_group)
            treatment = np.random.normal(true_effect, std, n_per_group)
            
            # 计算 t 统计量
            diff = treatment.mean() - control.mean()
            se = std * np.sqrt(2 / n_per_group)
            t_stat = diff / se
            
            # 检验
            if abs(t_stat) > z_alpha:
                rejections += 1
        
        empirical_power = rejections / n_simulations
        theoretical_power = self.calculate_power(n_per_group, true_effect, std)
        
        return {
            'empirical_power': empirical_power,
            'theoretical_power': theoretical_power,
            'difference': abs(empirical_power - theoretical_power),
            'n_simulations': n_simulations
        }


# 测试我们的实现
print("🔨 测试从零实现的 PowerAnalyzer")
print("=" * 60)

analyzer = PowerAnalyzer(alpha=0.05, power=0.8)

# 测试 1: 样本量计算
print("\n测试 1: 连续变量样本量计算")
result = analyzer.sample_size_continuous(effect_size=0.2, std=1.0)
print(f"效应大小: {result['effect_size']}")
print(f"标准差: {result['std']}")
print(f"每组样本量: {result['n_control']}")
print(f"z_alpha: {result['z_alpha']:.4f} (理论值 ≈ 1.96)")
print(f"z_beta: {result['z_beta']:.4f} (理论值 ≈ 0.84)")

# 测试 2: 功效计算
print("\n测试 2: 功效计算")
power = analyzer.calculate_power(n_per_group=400, effect_size=0.2, std=1.0)
print(f"样本量: 400/组")
print(f"效应: 0.2")
print(f"计算的功效: {power:.4f} (目标 0.8)")

# 测试 3: MDE 计算
print("\n测试 3: MDE 计算")
mde = analyzer.calculate_mde(n_per_group=400, std=1.0)
print(f"样本量: 400/组")
print(f"MDE: {mde:.4f} (应该接近 0.2)")

# 测试 4: 蒙特卡洛验证
print("\n测试 4: 蒙特卡洛模拟验证")
print("(这可能需要几秒钟...)")
sim_result = analyzer.simulate_experiment(
    n_per_group=400,
    true_effect=0.2,
    std=1.0,
    n_simulations=5000
)
print(f"理论功效: {sim_result['theoretical_power']:.4f}")
print(f"模拟功效: {sim_result['empirical_power']:.4f}")
print(f"差异: {sim_result['difference']:.4f}")
print("✅ 理论与模拟非常接近，说明我们的实现是正确的！")

# 测试 5: 比例检验
print("\n测试 5: 比例检验样本量计算")
result_prop = analyzer.sample_size_proportion(
    p_control=0.10,
    p_treatment=0.12
)
print(f"对照组转化率: {result_prop['p_control']:.1%}")
print(f"实验组转化率: {result_prop['p_treatment']:.1%}")
print(f"绝对提升: {result_prop['effect_size']:.2%}")
print(f"相对提升: {result_prop['relative_lift']:.1%}")
print(f"每组样本量: {result_prop['n_control']:,}")
print(f"总样本量: {result_prop['total_n']:,}")

---

## 🔨 Part 9: 从零实现 - PowerAnalyzer 类

在这一部分，我们将从零开始实现一个完整的功效分析工具，不依赖任何高级库（只用 numpy）。

这有助于深入理解每个步骤的数学原理。

---

## 🎤 Part 8: 高频面试题

### 题目 1: 样本量计算基础

**问题**: 如何计算 A/B 测试所需的样本量？关键参数有哪些？

**参考答案**:
- **关键参数**: α (显著性水平), Power (统计功效), 预期效应量 δ, 方差 σ²
- **公式**: $n = \frac{2\sigma^2 (z_{1-\alpha/2} + z_{1-\beta})^2}{\delta^2}$
- **步骤**:
  1. 确定业务上可接受的最小效应 (MDE)
  2. 从历史数据估计方差
  3. 设定 α=0.05, Power=0.8 (行业标准)
  4. 代入公式计算
- **注意事项**: 
  - 需要考虑分流比例（50:50 最优）
  - 方差估计要保守（避免低估）
  - 考虑流量损耗（用户流失、数据质量问题）

---

### 题目 2: Power 的含义

**问题**: 统计功效 (Power) 是什么？为什么通常设为 0.8？

**参考答案**:
- **定义**: Power = 1 - β = P(拒绝 H₀ | H₁ 为真)
- **含义**: 当真实效应存在时，我们能检测出来的概率
- **为什么是 0.8**:
  - 行业惯例（Cohen, 1988）
  - 权衡 Type I 和 Type II 错误：α=0.05, β=0.2 意味着我们认为「误报」比「漏报」更严重 (1:4 的比例)
  - 更高的 Power (如 0.9) 需要显著更多样本，成本急剧上升
- **实际应用**: 
  - 核心指标: Power ≥ 0.8
  - 次要指标: Power ≥ 0.5-0.7 (可接受)
  - 高风险决策: Power ≥ 0.9

---

### 题目 3: MDE 的业务含义

**问题**: MDE (最小可检测效应) 在实际业务中如何应用？

**参考答案**:
- **定义**: 给定样本量和显著性水平，能检测出的最小效应
- **业务应用**:
  1. **设定目标**: 如果 MDE > 业务可接受的最小提升，说明样本量不够
  2. **优先级排序**: 资源有限时，优先测试预期效应 > MDE 的实验
  3. **实验设计**: 如果基线方差很大，考虑 CUPED 等方差缩减技术降低 MDE
- **案例**: 
  - 转化率从 10% → 10.5% (相对提升 5%), 如果 MDE=2%, 可以检测
  - 但如果 MDE=3%, 就检测不出这个效应

---

### 题目 4: 偷看数据的问题

**问题**: 为什么不能在实验中途多次查看数据并提前停止？

**参考答案**:
- **问题**: 假阳性率膨胀（可能从 5% 膨胀到 20-30%）
- **原因**: 
  - 每次查看相当于做一次假设检验
  - 多次检验 = 多重比较问题
  - p-value 会随机波动，早期可能偶然 < 0.05
- **数学**: 
  - 查看 k 次，至少一次假阳性的概率: $1 - (1-\alpha)^k$
  - k=10, α=0.05: 约 40% 假阳性率！
- **解决方案**:
  - 预先设定固定样本量，到达后才分析
  - 如需中途分析，使用序贯分析 (Sequential Analysis)
  - Alpha Spending Function: 提前分配每次查看的 α

---

### 题目 5: 方差估计偏差的影响

**问题**: 如果用历史数据估计的方差有偏差，会有什么后果？

**参考答案**:
- **低估方差** (更常见):
  - 样本量计算偏小
  - 实际 Power < 目标 Power
  - 实验结束发现「不显著」，但浪费了时间
- **高估方差**:
  - 样本量过大
  - 浪费资源，实验周期过长
  - 但至少能保证 Power
- **应对措施**:
  - 使用最近的数据（避免业务变化）
  - 分层估计方差（不同用户群体可能差异大）
  - **保守估计**: 宁可高估方差，也不要低估
  - AA 测试验证方差估计

---

### 题目 6: 不平衡分流的场景

**问题**: 什么时候应该使用 90:10 而不是 50:50 分流？

**参考答案**:
- **90:10 的场景**:
  - **高风险实验**: 担心新版本损害用户体验
  - **灰度发布**: 先小流量验证，确认安全后扩大
  - **B端产品**: 客户数量少，不能大范围实验
- **代价**:
  - Power 显著降低（约需 2.8 倍样本量）
  - 实验周期更长
- **最佳实践**:
  - 先 95:5 或 90:10 灰度几天，验证关键指标
  - 确认无重大问题后，扩大到 50:50 完成统计检验
- **公式**: 方差 $\propto \frac{1}{p(1-p)n}$, p=0.5 时最小

---

### 题目 7: 多重比较校正

**问题**: 如果同时测试 10 个指标，如何控制整体假阳性率？

**参考答案**:
- **问题**: FWER (Family-Wise Error Rate) 膨胀
  - 单个指标 α=0.05
  - 10 个指标至少一个假阳性: $1-(1-0.05)^{10} \approx 40\%$
- **Bonferroni 校正**:
  - 设定每个检验的 α = 0.05 / k (k 为检验数量)
  - 保守，但简单
- **FDR (False Discovery Rate)**:
  - 控制假阳性占所有阳性的比例
  - Benjamini-Hochberg 方法
  - 比 Bonferroni 宽松，Power 更高
- **分层策略**:
  - **主要指标** (1-2 个): 不校正，α=0.05
  - **次要指标**: Bonferroni 或 FDR 校正
  - **护栏指标**: 单独检验，确保不恶化

---

### 题目 8: 样本量与效应量的关系

**问题**: 为什么效应量减半，所需样本量会翻 4 倍？

**参考答案**:
- **公式**: $n \propto \frac{1}{\delta^2}$
- **直觉解释**:
  - 效应量减半 → 信噪比减半
  - 为了维持同样的检测能力，需要更多样本来「放大信号」
  - 标准误 $SE \propto \frac{1}{\sqrt{n}}$，要使 SE 减半，n 需要翻 4 倍
- **数值例子**:
  - δ=0.2, σ=1: n ≈ 393
  - δ=0.1, σ=1: n ≈ 1,571 (约 4 倍)
- **业务启示**: 
  - 不要追求过小的效应（投入产出比低）
  - 优先测试预期效应大的策略

---

### 题目 9: 连续变量 vs 二元变量

**问题**: 转化率实验，应该用转化率 (0/1) 还是转化金额 (连续变量) 作为主要指标？

**参考答案**:
- **转化率 (二元变量)**:
  - 优点: 解释性强，业务直观
  - 缺点: 信息损失（忽略了金额差异）
  - 方差: p(1-p)，在 p=0.5 时最大
- **转化金额 (连续变量)**:
  - 优点: 包含更多信息，方差可能更小 → Power 更高
  - 缺点: 分布可能有偏（极值影响大）
  - 处理: 可以取对数或 Winsorize
- **最佳实践**:
  - **主要指标**: 用业务最关心的（通常是转化率或 GMV）
  - **辅助指标**: 两者都看
  - **CUPED**: 用历史金额作为协变量，提高转化率实验的 Power

---

### 题目 10: 实际工作中的沟通

**问题**: PM 说「我们只有 10,000 用户可以参与实验」，但计算出需要 20,000 人。怎么办？

**参考答案**:
- **方案 1: 降低 Power**
  - 从 0.8 降到 0.6-0.7
  - 权衡: 更可能漏报，但至少能做实验
- **方案 2: 放宽 MDE**
  - 只检测更大的效应
  - 明确告知 PM: 小于 X% 的提升检测不出来
- **方案 3: 延长实验时间**
  - 如果是流量限制，可以延长实验周期累积样本
- **方案 4: CUPED 等方差缩减技术**
  - 利用历史数据，可能减少 30-50% 样本量需求
- **方案 5: 分阶段实验**
  - 先小流量验证方向，确认有效后再扩大样本精确估计
- **沟通关键**:
  - 用业务语言: 「如果只有 10,000 人，我们只能检测到 X% 以上的提升」
  - 展示风险: 「有 40% 的概率漏掉真实效应」
  - 提供方案: 列出上述选项，让 PM 权衡

---

## 📋 总结

### 核心公式

**连续结果:**
$$n = \frac{2\sigma^2 (z_{1-\alpha/2} + z_{1-\beta})^2}{\delta^2}$$

**二元结果:**
$$n = \frac{2 \cdot \bar{p}(1-\bar{p}) \cdot (z_{1-\alpha/2} + z_{1-\beta})^2}{(p_B - p_A)^2}$$

### 关键参数

| 参数 | 典型值 | 说明 |
|------|--------|------|
| α (显著性水平) | 0.05 | 第一类错误率 |
| Power (功效) | 0.80 | 1 - 第二类错误率 |
| MDE | 业务决定 | 最小可检测效应 |

### 样本量影响因素

| 因素 | 与样本量的关系 |
|------|----------------|
| 效应大小 ↑ | 样本量 ↓ (平方关系) |
| 方差 ↑ | 样本量 ↑ (平方关系) |
| Power ↑ | 样本量 ↑ |
| α ↓ | 样本量 ↑ |

### 最佳实践清单

- ✅ **实验前**做功效分析
- ✅ 使用历史数据估计方差
- ✅ 保守估计预期效应
- ✅ 预设样本量，不要偷看
- ✅ 如需中途分析，使用序贯方法

---

**「不计算样本量的实验，就像不带地图的旅行——可能会迷路。」**

🎉 恭喜完成样本量计算与功效分析！